In [ ]:
!git clone https://github.com/advimman/lama.git
%cd lama

# 🔹 Force older compatible pip and dependencies
!pip install -U pip==23.3
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install omegaconf==2.3.0 pytorch-lightning==1.9.0 easydict==1.10
!pip install scikit-image==0.19.3 opencv-python-headless==4.8.1.78
!pip install tqdm numpy pyyaml

# 🔹 Download pretrained model weights
!wget -q https://huggingface.co/advimman/lama/resolve/main/big-lama.zip
!unzip -q big-lama.zip -d models


Cloning into 'lama'...
remote: Enumerating objects: 478, done.
remote: Counting objects: 100% (323/323), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 478 (delta 176), reused 128 (delta 128), pack-reused 155 (from 1)
Receiving objects: 100% (478/478), 8.84 MiB | 25.13 MiB/s, done.
Resolving deltas: 100% (194/194), done.
/content/lama/lama
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://download.pytorch.org/whl/cu118
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 49.1 MB/s eta 0:00:00
  Created wheel for easydict: filename=easydict-1.10-py3-none-any.whl size=6492 sha256=5d6edbb29f3baabc0e26af270ac42395bed173d795da340f321626a4509a3d2d


In [ ]:
# ✅ Colab: System + Python packages (run this cell)
# Takes a few minutes. If you don't want LaMa, keep the lama lines commented.

!apt-get update -qq
!apt-get install -y -qq poppler-utils ghostscript tesseract-ocr libtesseract-dev tesseract-ocr-urd

# ✅ Python packages
!pip install -q pymupdf==1.22.5 opencv-python pillow numpy pytesseract ocrmypdf

# ✅ OPTIONAL / HIGHLY RECOMMENDED for LaMa advanced inpainting (GPU recommended)
# Uncomment the next 2 lines if you want to enable LaMa support
# !pip install -q torch torchvision --extra-index-url https://download.pytorch.org/whl/cu118
# !pip install -q lama-cleaner


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package fonts-droid-fallback.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../00-fonts-droid-fallback_1%3a6.0.1r16-1.1build1_all.deb ...
Unpacking fonts-droid-fallback (1:6.0.1r16-1.1build1) ...
Selecting previously unselected package poppler-data.
Preparing to unpack .../01-poppler-data_0.4.11-1_all.deb ...
Unpacking poppler-data (0.4.11-1) ...
Selecting previously unselected package fonts-noto-mono.
Preparing to unpack .../02-fonts-noto-mono_20201225-1build1_all.deb ...
Unpacking fonts-noto-mono (20201225-1build1) ...
Selecting previously unselected package fonts-urw-base35.
Preparing to unpack .../03-fonts-urw-base35_20200910-1_all.deb ...
Unpacking fonts-urw-base35 (20200910-1) ...
Selecting previously unselected pac

In [ ]:
# Parameters - change if you want
DPI = 150               # 150 recommended for small size; use 200-300 for more clarity
JPEG_QUALITY = 70       # 60-80: smaller size vs clarity
USE_LAMA = False        # Set True to use lama-cleaner (requires GPU + lama installed)
LAMA_DEVICE = "cuda"    # "cuda" or "cpu" depending on runtime

# HSV thresholds for watermark detection (tune if needed)
HSV_BLUE_LOW = (90, 60, 40)
HSV_BLUE_HIGH = (140, 255, 255)
HSV_RED1_LOW = (0, 80, 40)
HSV_RED1_HIGH = (10, 255, 255)
HSV_RED2_LOW = (160, 80, 40)
HSV_RED2_HIGH = (180, 255, 255)

# Morphology / inpaint params
MORPH_KERNEL = (11, 11)
DILATE_ITER = 1
INPAINT_RADIUS = 3


In [ ]:
# Core pipeline functions
import fitz   # PyMuPDF
from PIL import Image
import io, os, cv2, numpy as np, glob, sys, subprocess

def pdf_to_images(pdf_path, dpi=DPI, fmt="jpg", quality=JPEG_QUALITY, tmp_dir="pages_in"):
    os.makedirs(tmp_dir, exist_ok=True)
    doc = fitz.open(pdf_path)
    img_paths = []
    for i, page in enumerate(doc):
        mat = fitz.Matrix(dpi/72, dpi/72)
        pix = page.get_pixmap(matrix=mat, alpha=False)
        img_bytes = pix.tobytes(fmt)
        p = os.path.join(tmp_dir, f"page_{i+1:03d}.jpg")
        with open(p, "wb") as f:
            f.write(img_bytes)
        img_paths.append(p)
    doc.close()
    return img_paths

def create_mask_for_watermark_bgr(bgr_image):
    hsv = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)
    lower_blue = np.array(HSV_BLUE_LOW); upper_blue = np.array(HSV_BLUE_HIGH)
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
    lower_red1 = np.array(HSV_RED1_LOW); upper_red1 = np.array(HSV_RED1_HIGH)
    lower_red2 = np.array(HSV_RED2_LOW); upper_red2 = np.array(HSV_RED2_HIGH)
    mask_red = cv2.bitwise_or(cv2.inRange(hsv, lower_red1, upper_red1), cv2.inRange(hsv, lower_red2, upper_red2))
    mask = cv2.bitwise_or(mask_blue, mask_red)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, MORPH_KERNEL)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=DILATE_ITER)
    return mask

def openCV_inpaint_save(in_path, out_path):
    img = cv2.imread(in_path)
    if img is None:
        raise ValueError("Failed to read image: " + in_path)
    mask = create_mask_for_watermark_bgr(img)
    nonzero = cv2.countNonZero(mask)
    if nonzero < 50:
        # no watermark detected: recompress
        pil = Image.open(in_path).convert("RGB")
        pil.save(out_path, format="JPEG", quality=JPEG_QUALITY, optimize=True)
        return False
    inpainted = cv2.inpaint(img, mask, INPAINT_RADIUS, flags=cv2.INPAINT_TELEA)
    pil = Image.fromarray(cv2.cvtColor(inpainted, cv2.COLOR_BGR2RGB))
    pil.save(out_path, format="JPEG", quality=JPEG_QUALITY, optimize=True)
    return True

# Optional LaMa wrapper (requires lama-cleaner installed)
LAMA_AVAILABLE = False
if 'lama_cleaner' in sys.modules:
    LAMA_AVAILABLE = True

def lama_inpaint_save(in_path, out_path, mask):
    # This function is a placeholder if you want to directly use lama-cleaner API.
    # The exact usage depends on the installed lama-cleaner version.
    # If USE_LAMA True but lama-cleaner not installed, fall back to OpenCV.
    raise NotImplementedError("LaMa wrapper must be implemented according to installed lama-cleaner API.")


In [ ]:
# Run conversion and cleaning
tmp_in_dir = "pages_in"
tmp_out_dir = "pages_out"
os.makedirs(tmp_in_dir, exist_ok=True)
os.makedirs(tmp_out_dir, exist_ok=True)

print("Converting PDF pages to JPEG images (DPI=%d) ..." % DPI)
img_paths = pdf_to_images(input_pdf, dpi=DPI, fmt="jpg", quality=JPEG_QUALITY, tmp_dir=tmp_in_dir)
print("Converted %d pages." % len(img_paths))

pages_cleaned = 0
for src in img_paths:
    name = os.path.basename(src)
    dst = os.path.join(tmp_out_dir, name)
    # If you want to use LaMa and it's available, call lama; otherwise OpenCV fallback
    if USE_LAMA and LAMA_AVAILABLE:
        # If you have lama-cleaner installed, implement lama_inpaint_save and call here
        try:
            # create mask for lama input
            img = cv2.imread(src)
            mask = create_mask_for_watermark_bgr(img)
            cleaned = False
            if cv2.countNonZero(mask) >= 50:
                # call your lama wrapper here (not implemented by default)
                # lama_inpaint_save(src, dst, mask)
                # For now fallback to OpenCV:
                cleaned = openCV_inpaint_save(src, dst)
            else:
                cleaned = openCV_inpaint_save(src, dst)
        except Exception as e:
            print("LaMa processing failed, falling back to OpenCV:", e)
            cleaned = openCV_inpaint_save(src, dst)
    else:
        cleaned = openCV_inpaint_save(src, dst)

    if cleaned:
        pages_cleaned += 1
    print(f"Processed {name} -> cleaned={cleaned}")

print(f"\nDone. Pages cleaned: {pages_cleaned} / {len(img_paths)}")


In [ ]:
from PIL import Image
out_images = sorted(glob.glob(os.path.join(tmp_out_dir, "page_*.jpg")))
if not out_images:
    raise SystemExit("No processed images found!")

print("Rebuilding compressed image-PDF ...")
pil_pages = [Image.open(p).convert("RGB") for p in out_images]
pil_pages[0].save(output_pdf_images_clean, save_all=True, append_images=pil_pages[1:], quality=JPEG_QUALITY, optimize=True)
print("Saved intermediate image-PDF:", output_pdf_images_clean)


In [ ]:
# Use ocrmypdf to create searchable PDF (requires tesseract). Urdu + English languages used here.
import subprocess, shlex
cmd = f"ocrmypdf --deskew --rotate-pages --output-type pdfa-1b -l eng+urd {shlex.quote(output_pdf_images_clean)} {shlex.quote(output_pdf_searchable)}"
print("Running OCRmyPDF (this may take a while)...")
print(cmd)
res = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
print("Return code:", res.returncode)
print(res.stdout)
print(res.stderr)
if res.returncode == 0:
    print("Searchable PDF saved at:", output_pdf_searchable)
else:
    print("OCRmyPDF had issues. You still have image-PDF:", output_pdf_images_clean)


In [ ]:
# If you want, I can generate a ready-to-download .ipynb file for you with these cells.
# Tell me if you want that; otherwise copy the cells into Colab manually.
print("If you want, I can prepare a downloadable .ipynb for you. Ask 'make notebook file' and I'll produce it.")
